In [3]:
import pandas as pd
import numpy as np
import scanpy as sc
from glob import glob
from scipy.sparse import issparse

import seaborn as sns
import matplotlib.pyplot as plt

sc.settings.verbosity = 3 
plt.rcParams['pdf.fonttype'] = 42
plt.rcParams['font.sans-serif'] = 'Arial'
sns.set_style('white', {'axes.grid' : False})

## bulk_herv_celltype

In [5]:
def adata_group(adata, groupby, layer=None, use_raw=False, min_cells=0):
    
    counts = adata.obs[groupby].value_counts() 
    indiv = counts[counts >= min_cells].index
    adata = adata[adata.obs[groupby].isin(indiv), :]
    index = adata.obs[groupby].unique()
    if use_raw:
        adata = adata.raw.to_adata()
    
    if layer is not None:
        csr = adata.layers[layer]
    else:
        csr = adata.X
    
    df = pd.DataFrame([np.array(csr[adata.obs[groupby]==x, :].sum(axis=0))[0, :] for x in index], index=index, columns=adata.var_names)
    
    bulk = sc.AnnData(df)
    
    age_dict = dict(zip(adata.obs[groupby], adata.obs['age']))
    sex_dict = dict(zip(adata.obs[groupby], adata.obs['sex']))
    #age_phase_dict = dict(zip(adata.obs[groupby], adata.obs['age_phase']))
    #age_stage_dict = dict(zip(adata.obs[groupby], adata.obs['age_stage']))
    bulk.obs['age'] = [age_dict[x] for x in bulk.obs_names]
    bulk.obs['sex'] = [sex_dict[x] for x in bulk.obs_names]
    #bulk.obs['age_phase'] = [age_phase_dict[x] for x in bulk.obs_names]
    #bulk.obs['age_stage'] = [age_stage_dict[x] for x in bulk.obs_names]
    bulk.obs[groupby] = bulk.obs_names.values
    return bulk

In [6]:
herv = sc.read_h5ad('herv_gene.h5ad')

In [7]:
herv.var

,mt,n_cells_by_counts,mean_counts,pct_dropout_by_counts,total_counts,n_cells,name,type
BX004987.1,False,1065,0.000889,99.911532,1070.0,1065,BX004987.1,gene
AC145212.1,False,1091,0.000909,99.909372,1094.0,1091,AC145212.1,gene
MAFIP,False,3781,0.003180,99.685918,3828.0,3781,MAFIP,gene
AC011043.1,False,656,0.000548,99.945507,660.0,656,AC011043.1,gene
AL354822.1,False,4500,0.003781,99.626192,4552.0,4500,AL354822.1,gene
...,...,...,...,...,...,...,...,...
THE1C_dup93-chrY,False,96,0.000082,99.992025,99.0,96,THE1C_dup93-chrY,herv
LTR16A_dup4-chrY,False,96,0.000081,99.992025,97.0,96,LTR16A_dup4-chrY,herv
HERVL-int_dup39-chrY,False,38,0.000032,99.996843,38.0,38,HERVL-int_dup39-chrY,herv
PRORY,False,30,0.000025,99.997508,30.0,30,PRORY,gene


In [8]:
herv.obs['indiv_ID_celltype'] = herv.obs['indiv_ID'].astype("str") +"_" + herv.obs['celltype'].astype("str")

In [9]:
bulk_herv = adata_group(herv,groupby = "indiv_ID_celltype")

In [10]:
sc.pp.normalize_total(bulk_herv,target_sum=1e6)
sc.pp.log1p(bulk_herv)

normalizing counts per cell
    finished (0:00:00)


In [11]:
bulk_herv.var['type'] = herv.var['type']

In [12]:
bulk_gene = bulk_herv[:,herv.var[herv.var['type'] == "gene"]['name']].copy()
bulk_herv = bulk_herv[:,herv.var[herv.var['type'] == "herv"]['name']].copy()

In [13]:
bulk_herv_df = pd.DataFrame(data=bulk_herv.X,index=bulk_herv.obs_names,columns=bulk_herv.var_names)
bulk_herv_df.to_csv("./expression/herv_bulk_celltype.csv",sep = "\t")

In [14]:
bulk_gene.write_h5ad('./gene_bulk_celltype.h5ad')
bulk_herv.write_h5ad('./herv_bulk_celltype.h5ad')

In [15]:
bulk_herv_df['name'] = bulk_herv_df.index
bulk_herv_df['celltype'] = bulk_herv_df['name'].apply(lambda x : x.split("_")[-1])
bulk_herv_df['indiv_ID'] = bulk_herv_df['name'].apply(lambda x : x.split("_")[0]+ "_" + x.split("_")[1])

In [16]:
bulk_herv_df.index = bulk_herv_df['indiv_ID']

In [18]:
for celltype in ['CD4-T','CD8-T','NK','Bcells','Myeloid']:
    bulk_herv_df[bulk_herv_df['celltype'].isin([celltype])].iloc[:,:-3].to_csv(f'./expression/{celltype}_herv.csv',sep = "\t")

## bulk_herv_indiv

In [4]:
herv = sc.read_h5ad('herv_gene.h5ad')

In [6]:
bulk_herv_indiv = adata_group(herv,groupby = "indiv_ID")
#bulk_gene_indiv = adata_group(adata_ref_gene,groupby = "indiv_ID",layer='counts')

In [7]:
sc.pp.normalize_total(bulk_herv_indiv,target_sum=1e6)
sc.pp.log1p(bulk_herv_indiv)

normalizing counts per cell
    finished (0:00:00)


In [9]:
bulk_herv_indiv.var['type'] = herv.var['type']

In [11]:
bulk_herv_indiv[:,herv.var['type'] == "herv"].write('herv_indiv.h5ad')
bulk_herv_indiv[:,herv.var['type'] == "gene"].write('gene_indiv.h5ad')

/home/zhufan/anaconda3/envs/cellphonedb/lib/python3.12/site-packages/anndata/_core/anndata.py:1301: ImplicitModificationWarning: Trying to modify attribute `.obs` of view, initializing view as actual.
  df[key] = c
/home/zhufan/anaconda3/envs/cellphonedb/lib/python3.12/site-packages/anndata/_core/anndata.py:1301: ImplicitModificationWarning: Trying to modify attribute `.obs` of view, initializing view as actual.
  df[key] = c


In [20]:
herv.var

,mt,n_cells_by_counts,mean_counts,pct_dropout_by_counts,total_counts,n_cells,name,type
BX004987.1,False,1065,0.000889,99.911532,1070.0,1065,BX004987.1,gene
AC145212.1,False,1091,0.000909,99.909372,1094.0,1091,AC145212.1,gene
MAFIP,False,3781,0.003180,99.685918,3828.0,3781,MAFIP,gene
AC011043.1,False,656,0.000548,99.945507,660.0,656,AC011043.1,gene
AL354822.1,False,4500,0.003781,99.626192,4552.0,4500,AL354822.1,gene
...,...,...,...,...,...,...,...,...
THE1C_dup93-chrY,False,96,0.000082,99.992025,99.0,96,THE1C_dup93-chrY,herv
LTR16A_dup4-chrY,False,96,0.000081,99.992025,97.0,96,LTR16A_dup4-chrY,herv
HERVL-int_dup39-chrY,False,38,0.000032,99.996843,38.0,38,HERVL-int_dup39-chrY,herv
PRORY,False,30,0.000025,99.997508,30.0,30,PRORY,gene


In [21]:
bulk_herv_indiv[:,herv.var['type'] == "herv"].X

ArrayView([[0.        , 0.23121397, 0.        , ..., 0.23121397,
            0.        , 0.23121397],
           [0.        , 0.        , 0.        , ..., 0.        ,
            0.        , 0.        ],
           [0.        , 0.        , 0.        , ..., 0.        ,
            0.        , 0.        ],
           ...,
           [0.        , 0.3128227 , 0.        , ..., 0.        ,
            0.        , 0.        ],
           [0.        , 0.        , 0.        , ..., 0.        ,
            0.        , 0.        ],
           [0.        , 0.        , 0.        , ..., 0.        ,
            0.        , 0.        ]], dtype=float32)

In [22]:
bulk_herv_indiv[:,herv.var['type'] == "herv"]

View of AnnData object with n_obs × n_vars = 981 × 41460
    obs: 'age', 'sex', 'indiv_ID'
    var: 'type'
    uns: 'log1p'

In [5]:
herv = sc.read_h5ad('herv.h5ad')

In [26]:
tmp = herv[:,herv.var['name'] == "RNASET2"].copy()

In [27]:
sc.pp.filter_cells(tmp,min_genes=1)

filtered out 725529 cells that have less than 1 genes expressed


In [28]:
tmp.obs['batch'].value_counts()

batch
GSM5899885    9525
GSM5899882    9138
GSM5899897    9049
GSM5899895    9023
GSM5899904    8786
              ... 
GSM5899931    4267
GSM5899908    4131
GSM5899937    3993
GSM5899930    3913
GSM5899944    3835
Name: count, Length: 75, dtype: int64

In [22]:
tmp.obs['batch'].value_counts()

batch
GSM5899925    21
GSM5899917    19
GSM5899946    16
GSM5899874    16
GSM5899882    15
              ..
GSM5899931     2
GSM5899893     2
GSM5899905     2
GSM5899944     1
GSM5899936     1
Name: count, Length: 74, dtype: int64

In [29]:
tmp.obs['celltype'].value_counts()

celltype
CD4-T      271967
CD8-T       62742
Bcells      61654
Myeloid     39753
NK          34835
gdT          2756
Plasma       2473
Mega         1252
HSC           864
Name: count, dtype: int64